In [1]:
#r "nuget: Jinaga, 0.8.2"
#r "nuget: Jinaga.Graphviz, 0.8.2"

Installed Packages Jinaga, 0.8.2 Jinaga.Graphviz, 0.8.2

# Invalidation

When receiving a graph of facts, we need to invalidate results based on the new information.
The specification inverse function assumes that we are receiving one fact at a time.
Adjust the algorithm to compute the difference in the projection after receiving a graph of facts.

Take this example model:

In [2]:
using Jinaga;

[FactType("Corporate.Company")]
record Company(string identifier) {}

[FactType("Corporate.City")]
record City(string name) {}

[FactType("Corporate.Office")]
record Office(Company company, City city)
{
    public Condition IsClosed => new Condition(facts =>
        facts.Any<OfficeClosure>(closure => closure.office == this)
    );
}

[FactType("Corporate.Office.Name")]
record OfficeName(Office office, string value, OfficeName[] prior) {}

[FactType("Corporate.Office.Closure")]
record OfficeClosure(Office office, DateTime closureDate) {}

We want to run this specification:

In [3]:
var officesInCompany = Given<Company>.Match((company, facts) =>
    from office in facts.OfType<Office>()
    where office.company == company
    where !office.IsClosed

    select new
    {
        office,
        names = facts.Observable(
            from name in facts.OfType<OfficeName>()
            where name.office == office &&
                !facts.Any<OfficeName>(next => next.prior.Contains(name))
            select name.value
        )
    }
);

Two of the inverses respond to new offices and new names:

In [9]:
var inverses = officesInCompany.ComputeInverses();
var officeInverse = inverses.Single(i => i.InverseSpecification.Given.Any(g => g.Name == "office"));
var officeNameInverse = inverses.Single(i => i.InverseSpecification.Given.Any(g => g.Name == "name"));

officeInverse.InverseSpecification.ToDescriptiveString().Display();
officeNameInverse.InverseSpecification.ToDescriptiveString().Display();

(office: Corporate.Office) {
    company: Corporate.Company [
        company = office->company: Corporate.Company
    ]
} => {
    names = {
        name: Corporate.Office.Name [
            name->office: Corporate.Office = office
            !E {
                next: Corporate.Office.Name [
                    next->prior: Corporate.Office.Name = name
                ]
            }
        ]
    } => name.value
    office = office
}


(name: Corporate.Office.Name) {
    office: Corporate.Office [
        office = name->office: Corporate.Office
        !E {
            closure: Corporate.Office.Closure [
                closure->office: Corporate.Office = office
            ]
        }
    ]
    company: Corporate.Company [
        company = office->company: Corporate.Company
    ]
} => name.value


When we receive a graph containing offices and names, we run those two inverses.

Suppose we receive this graph:

In [10]:
var j = JinagaClient.Create();

var contoso = await j.Fact(new Company("contoso"));
var dallas = await j.Fact(new City("Dallas"));
var dallasOffice = await j.Fact(new Office(contoso, dallas));
var dallasOfficeName1 = await j.Fact(new OfficeName(dallasOffice, "Dallas One", new OfficeName[0]));
var dallasOfficeName2 = await j.Fact(new OfficeName(dallasOffice, "Dallas Two", new OfficeName[] { dallasOfficeName1 }));
var dallasOfficeName3 = await j.Fact(new OfficeName(dallasOffice, "Dallas Three", new OfficeName[] { dallasOfficeName2 }));

Jinaga.Graphviz.Renderer.RenderFacts(dallasOffice, dallasOfficeName1, dallasOfficeName2, dallasOfficeName3)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 9.0.0 (20230911.1827)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- M2BrSspxPmU9hZq/0OGXCKIIFuS/wVhjbv2TaQYAL3yodutMRzqvtGC9IFM5/0WQ7YxrpHAER7zAfRUMAfQ7Cw== -->
 
 M2BrSspxPmU9hZq/0OGXCKIIFuS/wVhjbv2TaQYAL3yodutMRzqvtGC9IFM5/0WQ7YxrpHAER7zAfRUMAfQ7Cw== 
 
 Corporate.Company 
 
 identifier 
 
 contoso 
 
<!-- +8MGIQ5sVC+YQ5s72kKOj2uBaCeH+DjHh6T6LJ+p6/UHqDZsFXIIxdi1/qxl4vQV7ySckTl+0dlo8OPPAiuZHA== -->
 
 +8MGIQ5sVC+YQ5s72kKOj2uBaCeH+DjHh6T6LJ+p6/UHqDZsFXIIxdi1/qxl4vQV7ySckTl+0dlo8OPPAiuZHA== 
 
 Corporate.City 
 
 name 
 
 Dallas 
 
<!-- NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== -->
 
 NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== 
 
 Corporate.Office 
 
 
<!-- NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg==->M2BrSspxPmU9hZq/0OGXCKIIFuS/wVhjbv2TaQYAL3yodutMRzqvtGC9IFM5/0WQ7YxrpHAER7zAfRUMAfQ7Cw== -->
 
 NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg==->M2BrSspxPmU9hZq/0OGXCKIIFuS/wVhjbv2TaQYAL3yodutMRzqvtGC9IFM5/0WQ7YxrpHAER7zAfRUMAfQ7Cw== 
 
 
 company 
 
<!-- NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg==->+8MGIQ5sVC+YQ5s72kKOj2uBaCeH+DjHh6T6LJ+p6/UHqDZsFXIIxdi1/qxl4vQV7ySckTl+0dlo8OPPAiuZHA== -->
 
 NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg==->+8MGIQ5sVC+YQ5s72kKOj2uBaCeH+DjHh6T6LJ+p6/UHqDZsFXIIxdi1/qxl4vQV7ySckTl+0dlo8OPPAiuZHA== 
 
 
 city 
 
<!-- ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw== -->
 
 ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw== 
 
 Corporate.Office.Name 
 
 value 
 
 Dallas One 
 
 
<!-- ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw==->NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== -->
 
 ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw==->NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== 
 
 
 office 
 
<!-- rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA== -->
 
 rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA== 
 
 Corporate.Office.Name 
 
 value 
 
 Dallas Two 
 
 
<!-- rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA==->NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== -->
 
 rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA==->NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== 
 
 
 office 
 
<!-- rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA==->ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw== -->
 
 rPGto1W/tYPgCTCNtaFPvW27fWAED+ddA/1uONtxgN+FFY7yDp7v9A2TO/JxUoQXWBN5QgjJlhO9loYW7sYAzA==->ebslw+gV3XuzKrbKcGVIfgSA9kyZH0r1+aEUXzEVMDlZGYE8MH4My3jWUItytV+KccE5ajY+2u8X7jxpKew0zw== 
 
 
 prior 
 
<!-- I/MlU47IReQCH2VIPHL7YRKKMcG5v1BY7dOJiYrlTgZbiu8hy8e7dDDEfUXj1K6Kf6YUbXs6W1MJW7+9d94LJw== -->
 
 I/MlU47IReQCH2VIPHL7YRKKMcG5v1BY7dOJiYrlTgZbiu8hy8e7dDDEfUXj1K6Kf6YUbXs6W1MJW7+9d94LJw== 
 
 Corporate.Office.Name 
 
 value 
 
 Dallas Three 
 
 
<!-- I/MlU47IReQCH2VIPHL7YRKKMcG5v1BY7dOJiYrlTgZbiu8hy8e7dDDEfUXj1K6Kf6YUbXs6W1MJW7+9d94LJw==->NLr4dISvvUu1kh+yMwtiY3MdFNWMTUuh3e1AqE9t8FjYa/qqRAnMQoRt6doy/BdqbT6lylm38n8KR0hveudHIg== -->
 
 I/MlU47IReQCH2VIPHL7YRKKMcG5v1BY7dOJiYrlTgZbiu8hy8e7dDDEfUXj1K6Kf6YUbXs6W1MJW7+9d94LJw==->NLr4dISvvUu1kh

We will notify in topological order.
The first inverse fires for the office:

In [13]:
var typedOfficeInverse = Given<Office>.Match((office, facts) =>
    from company in facts.OfType<Company>()
    where company == office.company

    select new
    {
        office,
        names = facts.Observable(
            from name in facts.OfType<OfficeName>()
            where name.office == office &&
                !facts.Any<OfficeName>(next => next.prior.Contains(name))
            select name.value
        )
    }
);

await j.Query(typedOfficeInverse, dallasOffice)

index value 0 { office = Office { company = Company { identifier = contoso }, city = City { name = Dallas }, IsClosed = Jinaga.Condition }, names = Jinaga.Observers.ImmutableObservableCollection`1[System.String] } office Office { company = Company { identifier = contoso }, city = City { name = Dallas }, IsClosed = Jinaga.Condition } company Company { identifier = contoso } identifier contoso city City { name = Dallas } name Dallas IsClosed Jinaga.Condition Body facts => facts.Any(closure => (closure.office == Office { company = Company { identifier = contoso }, city = City { name = Dallas }, IsClosed = Jinaga.Condition })) Type System.Func<Jinaga.Repository.FactRepository,System.Boolean> NodeType Lambda Parameters index value 0 facts Name <null> Body facts.Any(closure => (closure.office == Office { company = Company { identifier = contoso }, city = City { name = Dallas }, IsClosed = Jinaga.Condition })) ArgumentCount 1 NodeType Call Type System.Boolean Method Boolean Any[OfficeClosure](System.Linq.Expressions.Expression`1[System.Func`2[Submission#2+OfficeClosure,System.Boolean]]) Object facts Arguments [ closure => (closure.office == Office { company = Company { identifier = contoso }, city = City { name = Dallas }, IsClosed = Jinaga.Condition }) ] CanReduce False ReturnType System.Boolean TailCall False CanReduce False names [ Dallas Three ] (values) [ Dallas Three ]

We notify the office.
Then we notify for the name -- Dallas Three.

The second fires for the names:

In [14]:
var typedNameInverse = Given<OfficeName>.Match((name, facts) =>
    from office in facts.OfType<Office>()
    where office == name.office
    where !office.IsClosed
    from company in facts.OfType<Company>()
    where company == office.company

    select name.value
);

await Task.WhenAll(
    j.Query(typedNameInverse, dallasOfficeName1),
    j.Query(typedNameInverse, dallasOfficeName2),
    j.Query(typedNameInverse, dallasOfficeName3)
)

index,value
0,[ Dallas One ]
1,[ Dallas Two ]
2,[ Dallas Three ]


Now we notify for Dallas One and Dallas Two.
We try to notify for Dallas Three, but it is discarded as a duplicate.

The final value is Dallas Two.